In [ ]:
ipython = get_ipython()
colab   = 'google.colab' in str(ipython)
print(ipython)

if colab:
  from google.colab import drive
  drive.mount('/content/gdrive')
  %cd '/content/gdrive/My Drive/Colab Notebooks'

!ls "./"

def MODULE_FROM_NOTEBOOK(target, source, *keys): 
    from json import load
    with open(source) as notebook:
        data = load(notebook)
        with open(target,'w') as module:
          for cell in data['cells']:
              lines = cell['source']
              if type(lines) == str:
                  lines = lines.split('\n')
              line0  = (lines or [''])[0].strip()
              haskey = lambda key: line0 == f'### {key} ###'
              if cell['cell_type'] == 'code' and any(map(haskey,keys)):
                  code = '\n'.join(lines)
                  module.write(code)
                  module.write('\n')
                     
MODULE_FROM_NOTEBOOK('./ttt_tactics.py',
                     './ttt-tactics.ipynb',
                     'HEAD','GAME','DATA','MODEL','SEARCH')

from ttt_tactics import *

ipynb = 'ttt-tactics-play'


Mounted at /content/gdrive
/content/gdrive/My Drive/Colab Notebooks
'!!!.ipynb'    ttt-tactics	        ttt_tactics.py
 __pycache__   ttt-tactics.ipynb        ttt-tactics-test.ipynb
 ttt-3.ipynb   ttt-tactics-play.ipynb   ttt-tactics-train.ipynb
[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 6178409667514218751
,
 name: "/device:XLA_CPU:0"
device_type: "XLA_CPU"
memory_limit: 17179869184
locality {
}
incarnation: 17921239274907681526
physical_device_desc: "device: XLA_CPU device"
]
tensorflow: 2.3.0
DIR = ./ttt-tactics
MODULE =  ttt_tactics
<function savesamples at 0x7f12a0e65730>
<function loadsamples at 0x7f12a0e65598>
<class 'ttt_tactics.AlphaZeroModel'>
<function AlphaZeroPolicy at 0x7f12a0e65b70>


In [ ]:
### SELF-PLAY ###

model = AlphaZeroModel()
# model.save(f'{DIR}/model-000.h5')
iters = 10_000
while iters > 0:
    iters -= 1

    datas = sortedfiles(f'{DIR}/data-*.tfrec')
    if len(datas) > 50:
        print('removing', datas[0])
        os.remove(datas[0])

    models = sortedfiles(f'{DIR}/model-*.h5')
    model.load(models[-1])

    pi = AlphaZeroPolicy(model, times=100, cp=2, tau=2)
    samples = samplegames({1:pi,2:pi}, iters=100)
    print('samples:', len(samples))

    path = f'{DIR}/data-{round(time())}.tfrec'
    savesamples(path, samples)
    MTIME(path)      

    NOTIFY(f'data ready, len={len(samples)}, path={path}, left={iters}, shell={str(ipython)}', title=ipynb, priority=-1)   


./ttt-tactics/model-008.h5 Oct 10 2020 13:18:11
samplegames 1:searchpi(model-008,times=100,cp=2,tau=2) vs 2:searchpi(model-008,times=100,cp=2,tau=2)
Instructions for updating:
Do not pass `graph_parents`.  They will  no longer be used.
Instructions for updating:
Do not call `graph_parents`.
100/100 [==============================] - 3650s 37s/step - episode: 81.0000 - agent: 0.4100 - rival: 0.3400 - draw: 0.2500
samples: 7226
./ttt-tactics/data-1602403072.0.tfrec Oct 11 2020 08:08:25
./ttt-tactics/model-008.h5 Oct 10 2020 13:18:11
samplegames 1:searchpi(model-008,times=100,cp=2,tau=2) vs 2:searchpi(model-008,times=100,cp=2,tau=2)
100/100 [==============================] - 3539s 35s/step - episode: 72.0000 - agent: 0.3800 - rival: 0.4600 - draw: 0.1600
samples: 6942
./ttt-tactics/data-1602407296.0.tfrec Oct 11 2020 09:18:11
./ttt-tactics/model-008.h5 Oct 10 2020 13:18:11
samplegames 1:searchpi(model-008,times=100,cp=2,tau=2) vs 2:searchpi(model-008,times=100,cp=2,tau=2)
100/100 [=======